In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [3]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
df2 = df.drop(columns=['date', 'lights'])

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
scaler = MinMaxScaler()

In [7]:
normalised_df = pd.DataFrame(scaler.fit_transform(df2),
                           columns=df2.columns)

In [8]:
normalised_df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [9]:
featured_df = normalised_df.drop(columns=['Appliances'])
target_df = normalised_df['Appliances']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_test, y_train, y_test = train_test_split(featured_df, target_df,
                                                   test_size=0.3, 
                                                    random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [14]:
prediction = linear_model.predict(x_test)

# Question 18

In [15]:
from sklearn.metrics import mean_absolute_error

In [16]:
mae = mean_absolute_error(y_test, prediction)
mae

0.05013362397742957

# Question 19

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
import numpy as np

In [19]:
rmse = np.sqrt(mean_squared_error(y_test, prediction))
round(rmse, 3)            #the lower the better

0.088

In [20]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

In [21]:
# compairing the effects of regulatisation

def get_weight_df(model, feat, col_name):
    """
    returns the weigth of every feature
    """
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weight_df = pd.DataFrame(weights).reset_index()
    weight_df.columns = ['Features', col_name]
    weight_df[col_name].round(3)
    return weight_df

In [22]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [23]:
linear_model_weights = get_weight_df(linear_model, x_train, 
                                     'Linear_Model_Weight')
ridge_weight_df = get_weight_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weight_df = get_weight_df(lasso_reg, x_train, 'Lasso_Weight')

In [24]:
final_weights = pd.merge(linear_model_weights, ridge_weight_df, 
                         on='Features')
final_weights = pd.merge(final_weights, lasso_weight_df, on='Features')

# Question 24

In [25]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


# Question 25

In [26]:
lasso_pred = lasso_reg.predict(x_test)

In [27]:
rmse_lasso = mean_squared_error(y_test, lasso_pred)
rmse_lasso

0.008757535449401907